In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical

from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.utils import class_weight

# import seaborn as sns
# import matplotlib.pylot as plt
from sklearn.metrics import precision_recall_fscore_support

In [29]:
data = pd.read_csv('training_data/eurusd_final_dataset.csv')

# I am dropping because I don't want to use these features in the model currently
X = data.drop(['label', 'Date_Time', 'hour', 'dayofweek', 'mins_into_m15', 'frac_into_m15'], axis=1)
y = data['label']

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify=y, test_size=0.2, random_state=42)

In [33]:
# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes=5)
y_test_cat = to_categorical(y_test, num_classes=5)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')
])

# Use the integer version of the labels here
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Convert to a dict if needed
class_weights = dict(enumerate(class_weights))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train_cat, class_weight=class_weights, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

Epoch 1/20
3837/3837 [==============================] - 18s 4ms/step - loss: 1.3467 - accuracy: 0.5228 - val_loss: 1.1866 - val_accuracy: 0.5386
Epoch 2/20
3837/3837 [==============================] - 17s 4ms/step - loss: 1.3235 - accuracy: 0.5433 - val_loss: 1.1862 - val_accuracy: 0.5391
Epoch 3/20
3837/3837 [==============================] - 17s 4ms/step - loss: 1.3177 - accuracy: 0.5461 - val_loss: 1.1504 - val_accuracy: 0.5503
Epoch 4/20
3837/3837 [==============================] - 17s 5ms/step - loss: 1.3141 - accuracy: 0.5454 - val_loss: 1.1737 - val_accuracy: 0.5368
Epoch 5/20
3837/3837 [==============================] - 17s 4ms/step - loss: 1.3116 - accuracy: 0.5467 - val_loss: 1.1710 - val_accuracy: 0.5512
Epoch 6/20
3837/3837 [==============================] - 17s 4ms/step - loss: 1.3090 - accuracy: 0.5458 - val_loss: 1.1620 - val_accuracy: 0.5509
Epoch 7/20
3837/3837 [==============================] - 17s 5ms/step - loss: 1.3082 - accuracy: 0.5461 - val_loss: 1.1455 - val_ac

In [ ]:
# Predict
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Metrics
print("Keras Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
#sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
#plt.xlabel("Predicted")
#plt.ylabel("True")
#plt.show()

# Per-class Precision/Recall/F1
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)
for i in range(5):
    print(f"Class {i} — Precision: {prec[i]:.2f}, Recall: {rec[i]:.2f}, F1: {f1[i]:.2f}")
    

1066/1066 [==============================] - 3s 3ms/step
Keras Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.75      0.75     16072
           1       0.50      0.40      0.44     12110
           2       0.26      0.33      0.29      3738
           3       0.16      0.31      0.21      1243
           4       0.24      0.28      0.26       941

    accuracy                           0.55     34104
   macro avg       0.38      0.41      0.39     34104
weighted avg       0.57      0.55      0.56     34104

Confusion Matrix:
Class 0 — Precision: 0.75, Recall: 0.75, F1: 0.75
Class 1 — Precision: 0.50, Recall: 0.40, F1: 0.44
Class 2 — Precision: 0.26, Recall: 0.33, F1: 0.29
Class 3 — Precision: 0.16, Recall: 0.31, F1: 0.21
Class 4 — Precision: 0.24, Recall: 0.28, F1: 0.26


In [ ]:
# Simulated trade logic example: TP = +2, SL = -1 per class prediction
profit_rules = {0: 2, 1: -1, 2: 2, 3: -1, 4: 0}

# Assume true class is the correct "direction" — reward if prediction matches
profits = [profit_rules[p] if p == t else -1 for p, t in zip(y_pred, y_test)]
total_profit = sum(profits)

print(f"Simulated Profit: {total_profit}")